In [ ]:
AOI = 'POLYGON ((-85.95552469759967 40.59907889076951, -85.8719838031499 39.71944241391614, -84.68029751467536 39.69732864773826, -84.7011827382878 40.61013577385845, -85.95552469759967 40.59907889076951))'
START_DATE = "2020-05-01"
END_DATE = "2020-08-30"

In [ ]:
import shapely.wkt
import time
import os
import shutil
import sys
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
sys.path.append("code/")
from code.prediction_pipeline.run import run

In [ ]:
user = os.getenv('NB_USER')

In [ ]:
polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")
start_date = datetime.strptime(START_DATE, '%Y-%m-%d')
end_date = datetime.strptime(END_DATE, '%Y-%m-%d')

In [ ]:
CONFIG_PATH = f"/home/{user}/work/notebooks/pw/code/prediction_pipeline/config.yml"
prediction_file = run(CONFIG_PATH, aoi_filename, start_date, end_date)
os.remove(aoi_filename)

### Read prediction raster and mask with valid pixels 

In [ ]:
with rasterio.open(prediction_file) as src:
    img = src.read(1)
    mask = src.read_masks(1)
    profile = src.profile

### Scale class prediction raster and make it colored

In [ ]:
NUM_CLASSES = 6
mask = mask.astype(bool)
scaled = img.astype(np.float32) / NUM_CLASSES
scaled = (plt.cm.jet(scaled)[:,:,:-1] * 255)
scaled[mask[:,:,np.newaxis] & (scaled==0)] +=1
scaled = scaled.astype(np.uint8)
scaled[~mask] = 0

### Update raster profile and save results

In [ ]:
prediction_dir = os.path.dirname(prediction_file)

profile.update(
    count=3,
    compress='lzw'
)
colored_tif = os.path.join(prediction_dir, "colored_prediction.tif")
with rasterio.open(colored_tif, 'w', **profile) as dst:
    dst.update_tags(start_date=START_DATE, 
                    end_date=END_DATE, 
                    request_id=REQUEST_ID)
    for i in range(scaled.shape[-1]):
        dst.write(scaled[:,:,i], indexes=i+1)

In [ ]:
RESULTS_DIR = f"/home/{user}/work/results/pw/"
result_name = f"{REQUEST_ID}_{START_DATE}_{END_DATE}.tif"
os.makedirs(RESULTS_DIR, exist_ok=True)
dst_file = os.path.join(RESULTS_DIR, result_name)
os.rename(colored_tif, dst_file)